In [1]:
import cv2
import numpy as np
import torch
from pathlib import Path

from tools.darknet2pytorch import Darknet
from tools.torch_utils import do_detect

In [2]:
def readNamesFile(self, nf):
    with open(nf, 'r', encoding='utf-8') as f:
        names = [line.rstrip() for line in f.readlines()]
    self.names = {i: s for i, s in enumerate(names)}

In [3]:
model_name = 'yolov4'
model_dir = Path('models', model_name)
config_file = Path(model_dir, f'{model_name}.cfg')
weights_file = Path(model_dir, f'{model_name}.weights')
names_file = Path(model_dir, 'coco.names')  

net = Darknet(config_file).cuda().eval()
net.load_weights(weights_file)
net_size = (net.width, net.height)

with open(names_file, 'r') as f:
    classes = [line.rstrip() for line in f.readlines()]

print(f'<{model_name}> model loaded.')

<yolov4> model loaded.


In [4]:
import colorsys, random

def draw_bbox(image, bboxes, classes=classes, show_label=True):
    num_classes = len(classes)
    image_h, image_w, _ = image.shape
    hsv_tuples = [(1.0 * x / num_classes, 1., 1.) for x in range(num_classes)]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)), colors))

    random.seed(0)
    random.shuffle(colors)
    random.seed(None)

    for bbox in bboxes:
        x1 = int(bbox[0] * image_w)
        y1 = int(bbox[1] * image_h)
        x2 = int(bbox[2] * image_w)
        y2 = int(bbox[3] * image_h)
        score = bbox[4]
        obj = bbox[5]
        
        bbox_color = colors[obj]
        bbox_thick = int(0.6 * (image_h + image_w) / 600)
        c1, c2 = (x1, y1), (x2, y2)
        cv2.rectangle(image, c1, c2, bbox_color, bbox_thick)

        if show_label:
            fontScale = 0.5
            bbox_mess = '%s: %.2f' % (classes[obj], score)
            t_size = cv2.getTextSize(bbox_mess, 0, fontScale, thickness=bbox_thick // 2)[0]
            c3 = (c1[0] + t_size[0], c1[1] - t_size[1] - 3)
            cv2.rectangle(image, c1, (c3[0], c3[1]), bbox_color, -1) #filled
            cv2.putText(image, bbox_mess, (c1[0], c1[1] - 2), cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale, (0, 0, 0), bbox_thick // 2, lineType=cv2.LINE_AA)
    return image

In [7]:
data_dir = 'data'
image_path = Path(data_dir, 'kite.jpg')
image = cv2.cvtColor(cv2.imread(str(image_path)), cv2.COLOR_BGR2RGB)

bboxes = do_detect(net, cv2.resize(image, net_size), .6, .5)[0]

In [8]:
image_result = draw_bbox(image.copy(), bboxes)
cv2.imshow('demo', cv2.cvtColor(image_result, cv2.COLOR_BGR2RGB))
cv2.waitKey(0)
cv2.destroyAllWindows()